In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import itertools
import numpy as np
import math
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [3]:
class MatrixDataset(Dataset):
    def __init__(self, matrices, labels):
        self.matrices = matrices
        self.labels = labels

    def __len__(self):
        return len(self.matrices)

    def __getitem__(self, idx):
        matrix = self.matrices[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return matrix, label
    
class Prey_Net(nn.Module):
    def __init__(self):
        super(Prey_Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class Predator_Net(nn.Module):
    def __init__(self):
        super(Predator_Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def find_closest_cell(t, point, n):
    """
    t: torch tensor filled with 1s and 2s
    point: tuple (x,y) of the currently observed point
    n: number to search for (1 or 2)
    """
    closest_dist = math.inf
    closest_cell = None
    
    cells = [(i, j) for i in range(t.size()[0]) for j in range(t.size()[1]) if t[i][j] == n]
    
    for cell_point in cells:
        dist = abs(point[0] - cell_point[0]) + abs(point[1] - cell_point[1]) 
        if dist < closest_dist:
            closest_dist = dist
            closest_cell = cell_point
                    
    if closest_cell is None:
        return math.inf
    else:
        return closest_dist


def find_best_move(t, agent):
# Moves are enumerated as # Stand, Top, Right, Bottom, Left
    n = 1 if agent == 2 else 2
    center = (1,1) if agent == 1 else (2,2)

    if agent == 1:
        best_distance = -math.inf
    else:
        best_distance = math.inf

    best_moves = []
    for move in [ (center[0]-1, center[1], 1),  (center[0], center[1]+1, 2), (center[0]+1, center[1], 3), (center[0], center[1]-1, 4)]:
        if (agent == 1 and t[move[0]][move[1]] == 0) or (agent == 2 and t[move[0]][move[1]] != 2):
            new_distance = find_closest_cell(t, move[:2], n)
            
            if (agent == 1 and new_distance > best_distance) or (agent == 2 and new_distance < best_distance):
                best_distance = new_distance
                best_moves = [move[2]]
            elif new_distance == best_distance:
                best_moves.append(move[2])
      

    if len(best_moves) == 0:
        return [1, 0, 0, 0, 0]
      
    prob = 1/len(best_moves)
    prob_vec = [0]*5

    for i in best_moves:
        prob_vec[i] = prob

    return prob_vec


def infer(net, t):
    net.eval()

    with torch.no_grad():  # Disable gradient calculation to save memory and computation
        logits = net(t.unsqueeze(0).to(device))
        probabilities = torch.softmax(logits, dim=1)

        return [el.item() for el in probabilities[0]] 

# Prey training

In [4]:
# Create a list of all possible combinations of 0, 1, and 2
vals = [0, 1, 2]
combinations = list(itertools.product(vals, repeat=9))

# Filter out the combinations where the middle element is not 0
filtered_combinations = list(filter(lambda x: x[4] == 1, combinations))

# Reshape the filtered combinations to tensors of shape (3x3)
prey_tensors = list([torch.tensor(combination, dtype=torch.float32).reshape(3, 3) for combination in filtered_combinations])
prey_labels = [find_best_move(t, 1) for t in prey_tensors]  

In [5]:
batch_size = 32
dataset = MatrixDataset(prey_tensors, prey_labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

prey_net = Prey_Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(prey_net.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = prey_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 0.8118978008917235
Epoch: 2, Loss: 0.3172369461000255
Epoch: 3, Loss: 0.2859878096982692
Epoch: 4, Loss: 0.27354813743274187
Epoch: 5, Loss: 0.26234148960623854
Epoch: 6, Loss: 0.25397845402866315
Epoch: 7, Loss: 0.24411047152718257
Epoch: 8, Loss: 0.23947931804413936
Epoch: 9, Loss: 0.23676978093728782
Epoch: 10, Loss: 0.23474327862732908
Epoch: 11, Loss: 0.23451693986375247
Epoch: 12, Loss: 0.230169416558974
Epoch: 13, Loss: 0.2294472791808341
Epoch: 14, Loss: 0.22835649716290893
Epoch: 15, Loss: 0.23204635441737267
Epoch: 16, Loss: 0.23206566907555878
Epoch: 17, Loss: 0.23105917892699102
Epoch: 18, Loss: 0.22531313818847779
Epoch: 19, Loss: 0.2234322483178332
Epoch: 20, Loss: 0.2225502928390023
Epoch: 21, Loss: 0.22397468456076186
Epoch: 22, Loss: 0.22444628483787613
Epoch: 23, Loss: 0.22134648617899533
Epoch: 24, Loss: 0.2212209668580294
Epoch: 25, Loss: 0.2201975062119354
Epoch: 26, Loss: 0.22187820807533357
Epoch: 27, Loss: 0.21866945245675348
Epoch: 28, Loss: 0.2

In [6]:
# Example of inference
t = prey_tensors[123]
print(t)
print()
infer(prey_net, t)

tensor([[0., 0., 0.],
        [1., 1., 1.],
        [1., 2., 0.]])



[1.4566735728749336e-07,
 0.9999563694000244,
 3.737292217920185e-06,
 2.4639255080671774e-09,
 3.98547163058538e-05]

# Predator training

In [7]:
# # Create a list of all possible combinations of 0, 1, and 2
# vals = [0, 1, 2]
# combinations = list(itertools.product(vals, repeat=25))

# # Filter out the combinations where the middle element is not 0
# filtered_combinations = list(filter(lambda x: x[12] == 2, combinations))

# # Reshape the filtered combinations to tensors of shape (5x5)
# prey_tensors = list([torch.tensor(combination, dtype=torch.float32).reshape(5, 5) for combination in filtered_combinations])
# # prey_labels = [find_best_move(t, 1) for t in prey_tensors]  

In [8]:
num_tensors = 10000
tensor_shape = (5, 5)
probs = torch.tensor([0.9, 0.08, 0.02], dtype=torch.float32)

predator_tensors = []

for _ in range(num_tensors):
    # Generate a 5x5 tensor with elements drawn from the multinomial distribution
    random_tensor = torch.multinomial(probs, tensor_shape[0] * tensor_shape[1], replacement=True).reshape(tensor_shape)

    # Convert the generated tensor to torch.float32
    random_tensor = random_tensor.to(torch.float32)

    # Set the middle element to 2
    random_tensor[tensor_shape[0] // 2, tensor_shape[1] // 2] = 2

    # Append the generated tensor to the list
    predator_tensors.append(random_tensor)

predator_labels = [find_best_move(t, 2) for t in predator_tensors] 

In [9]:
batch_size = 32
dataset = MatrixDataset(predator_tensors, predator_labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

predator_net = Predator_Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(predator_net.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = predator_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 1.2025702664265618
Epoch: 2, Loss: 0.9329797387503969
Epoch: 3, Loss: 0.756704972574886
Epoch: 4, Loss: 0.6857070522948195
Epoch: 5, Loss: 0.6622354436796695
Epoch: 6, Loss: 0.6462246234805439
Epoch: 7, Loss: 0.6391736012868607
Epoch: 8, Loss: 0.6300538725936755
Epoch: 9, Loss: 0.6253231324898169
Epoch: 10, Loss: 0.621974328741098
Epoch: 11, Loss: 0.6163725848205555
Epoch: 12, Loss: 0.6133570658703582
Epoch: 13, Loss: 0.6122356690347385
Epoch: 14, Loss: 0.6107570408060908
Epoch: 15, Loss: 0.607594551465001
Epoch: 16, Loss: 0.604643281370687
Epoch: 17, Loss: 0.6031798463279066
Epoch: 18, Loss: 0.6013739837434726
Epoch: 19, Loss: 0.5987175793503039
Epoch: 20, Loss: 0.5986585041966301
Epoch: 21, Loss: 0.5959485289387809
Epoch: 22, Loss: 0.5950672165654338
Epoch: 23, Loss: 0.5947728794032392
Epoch: 24, Loss: 0.5943023912822858
Epoch: 25, Loss: 0.5926074691283436
Epoch: 26, Loss: 0.5915394584401347
Epoch: 27, Loss: 0.5897407093748879
Epoch: 28, Loss: 0.5901321776377888
Epoch

In [10]:
find_best_move(predator_tensors[123], 2)

[0, 0, 0, 1.0, 0]

In [11]:
# Example of inference
t = predator_tensors[123]
print(t)
print()
infer(predator_net, t)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])



[3.0500759936266215e-27,
 4.678115317346965e-08,
 8.619059371994808e-05,
 0.9995949864387512,
 0.0003187795227859169]